In [3]:

# ============================================================
# Cell 1: Dataset Inventory & Path Sanity (Memory-Safe)
# ============================================================

from pathlib import Path

print("🔍 MarketSentinel — Data Contract Validation")
print("📂 Running Cell 1: Dataset inventory & path sanity\n")

# PROJECT_ROOT = Path("D:/MarketSentinel/MarketSentinel")  # or absolute path
PROJECT_ROOT = Path.cwd().parent
DATASET_DIR = PROJECT_ROOT / "datasets"

print(f"Project root:   {PROJECT_ROOT}")
print(f"Datasets path:  {DATASET_DIR}\n")

# ---- Hard assertions ----
assert PROJECT_ROOT.exists(), "❌ Project root does not exist"
assert DATASET_DIR.exists(), "❌ datasets/ directory is missing"

REQUIRED_FILES = [
    "feature_schema.txt",
    "node_features.parquet",
    "edges_static.parquet",
    "gnn_sequences_train.h5",
    "gnn_sequences_val.h5",
    "gnn_sequences_test.h5",
]

missing = [f for f in REQUIRED_FILES if not (DATASET_DIR / f).exists()]
assert not missing, f"❌ Missing required dataset files: {missing}"

print("✅ Required dataset files present:")
for f in REQUIRED_FILES:
    print(f"  - {f}")

print("\n✅ Cell 1 PASSED — Dataset presence verified.")
print("➡️ Proceed to Cell 2: feature_schema.txt validation")


🔍 MarketSentinel — Data Contract Validation
📂 Running Cell 1: Dataset inventory & path sanity

Project root:   d:\MarketSentinel\MarketSentinel
Datasets path:  d:\MarketSentinel\MarketSentinel\datasets

✅ Required dataset files present:
  - feature_schema.txt
  - node_features.parquet
  - edges_static.parquet
  - gnn_sequences_train.h5
  - gnn_sequences_val.h5
  - gnn_sequences_test.h5

✅ Cell 1 PASSED — Dataset presence verified.
➡️ Proceed to Cell 2: feature_schema.txt validation


In [4]:
# ============================================================
# Cell 2: feature_schema.txt validation
# ============================================================

from pathlib import Path

print("🧪 Running Cell 2: feature_schema.txt validation\n")

PROJECT_ROOT = Path.cwd().parent
DATASET_DIR = PROJECT_ROOT / "datasets"
SCHEMA_PATH = DATASET_DIR / "feature_schema.txt"

# ---- Load schema ----
assert SCHEMA_PATH.exists(), "❌ feature_schema.txt does not exist"

with open(SCHEMA_PATH, "r") as f:
    raw_features = [line.rstrip("\n") for line in f.readlines()]

# ---- Hard validations ----
assert len(raw_features) > 0, "❌ feature_schema.txt is empty"

# No empty feature names
empty_features = [i for i, f in enumerate(raw_features) if f.strip() == ""]
assert not empty_features, f"❌ Empty feature names at lines: {empty_features}"

# No leading/trailing whitespace
whitespace_issues = [f for f in raw_features if f != f.strip()]
assert not whitespace_issues, (
    "❌ Feature names contain leading/trailing whitespace: "
    f"{whitespace_issues}"
)

# Uniqueness
duplicates = sorted(set([f for f in raw_features if raw_features.count(f) > 1]))
assert not duplicates, f"❌ Duplicate feature names found: {duplicates}"

# ---- Final report ----
num_features = len(raw_features)

print(f"✅ feature_schema.txt loaded successfully")
print(f"🔢 Total features: {num_features}")
print("📌 First 5 features:")
for f in raw_features[:5]:
    print(f"  - {f}")

print("📌 Last 5 features:")
for f in raw_features[-5:]:
    print(f"  - {f}")


🧪 Running Cell 2: feature_schema.txt validation

✅ feature_schema.txt loaded successfully
🔢 Total features: 263
📌 First 5 features:
  - atr_14
  - atr_14_rmean_60
  - atr_14_rstd_60
  - atr_14_rz_60
  - close_z_20
📌 Last 5 features:
  - volume_z_20
  - volume_z_20_rmean_60
  - volume_z_20_rstd_60
  - volume_z_20_rz_60
  - year


In [6]:
# ============================================================
# Cell 3A: FIX — enforce float32 on node_features.parquet
# ============================================================

from pathlib import Path
import pandas as pd
import numpy as np

print("🛠 Fixing node_features.parquet dtypes → float32\n")

PROJECT_ROOT = Path.cwd().parent
DATASET_DIR = PROJECT_ROOT / "datasets"

NODE_FEATURES_PATH = DATASET_DIR / "node_features.parquet"
SCHEMA_PATH = DATASET_DIR / "feature_schema.txt"

# Load schema
with open(SCHEMA_PATH, "r") as f:
    feature_schema = [line.rstrip("\n") for line in f.readlines()]

# Load parquet
df = pd.read_parquet(NODE_FEATURES_PATH)

# Cast features explicitly
for col in feature_schema:
    df[col] = df[col].astype(np.float32)

# Safety check
for col in feature_schema:
    assert df[col].dtype == np.float32, f"❌ Failed to cast {col} to float32"

# Rewrite parquet (overwrite is intentional)
df.to_parquet(NODE_FEATURES_PATH, index=False)

print("✅ node_features.parquet rewritten with float32 features")
print("➡️ Re-run Cell 3")


🛠 Fixing node_features.parquet dtypes → float32

✅ node_features.parquet rewritten with float32 features
➡️ Re-run Cell 3


In [7]:
# ============================================================
# Cell 3: node_features.parquet validation (memory-safe)
# ============================================================

from pathlib import Path
import numpy as np
import pandas as pd

print("🧪 Running Cell 3: node_features.parquet validation\n")

PROJECT_ROOT = Path.cwd().parent
DATASET_DIR = PROJECT_ROOT / "datasets"

NODE_FEATURES_PATH = DATASET_DIR / "node_features.parquet"
SCHEMA_PATH = DATASET_DIR / "feature_schema.txt"

assert NODE_FEATURES_PATH.exists(), "❌ node_features.parquet does not exist"
assert SCHEMA_PATH.exists(), "❌ feature_schema.txt does not exist"

# ---- Load feature schema ----
with open(SCHEMA_PATH, "r") as f:
    feature_schema = [line.rstrip("\n") for line in f.readlines()]

F = len(feature_schema)
assert F > 0, "❌ Feature schema is empty (should not happen if Cell 2 passed)"

# ---- Load parquet (column metadata first) ----
df = pd.read_parquet(NODE_FEATURES_PATH)

# ---- Column order validation ----
expected_columns = ["symbol", "date"] + feature_schema
actual_columns = list(df.columns)

assert actual_columns == expected_columns, (
    "❌ Column order mismatch in node_features.parquet\n"
    f"Expected: {expected_columns[:5]} ...\n"
    f"Actual:   {actual_columns[:5]} ..."
)

# ---- Dtype validation ----
assert df["symbol"].dtype == object, "❌ 'symbol' column must be string/object"
assert np.issubdtype(df["date"].dtype, np.datetime64), (
    "❌ 'date' column must be datetime64"
)

for col in feature_schema:
    assert df[col].dtype == np.float32, (
        f"❌ Feature '{col}' has dtype {df[col].dtype}, expected float32"
    )

# ---- Finiteness validation (features only) ----
feature_values = df[feature_schema].to_numpy()
assert np.isfinite(feature_values).all(), "❌ Non-finite values found in features"

# ---- Symbol sanity ----
assert df["symbol"].isnull().sum() == 0, "❌ Null symbols found"
unique_symbols = sorted(df["symbol"].unique())

# ---- Final report ----
print("✅ node_features.parquet structure is valid")
print(f"📐 Total rows: {len(df)}")
print(f"🔢 Feature count: {F}")
print(f"🔗 Unique symbols ({len(unique_symbols)}): {unique_symbols[:5]}{'...' if len(unique_symbols) > 5 else ''}")

print("\n✅ Cell 3 PASSED — Node feature table conforms to contract.")
print("➡️ Proceed to Cell 4: edges_static.parquet validation")


🧪 Running Cell 3: node_features.parquet validation

✅ node_features.parquet structure is valid
📐 Total rows: 257794
🔢 Feature count: 263
🔗 Unique symbols (66): ['AAPL', 'ABBV', 'AMZN', 'ASML', 'BA']...

✅ Cell 3 PASSED — Node feature table conforms to contract.
➡️ Proceed to Cell 4: edges_static.parquet validation


In [8]:
# ============================================================
# Cell 4: edges_static.parquet validation
# ============================================================

from pathlib import Path
import pandas as pd
import numpy as np

print("🧪 Running Cell 4: edges_static.parquet validation\n")

PROJECT_ROOT = Path.cwd().parent
DATASET_DIR = PROJECT_ROOT / "datasets"

EDGES_PATH = DATASET_DIR / "edges_static.parquet"
NODE_FEATURES_PATH = DATASET_DIR / "node_features.parquet"

assert EDGES_PATH.exists(), "❌ edges_static.parquet does not exist"
assert NODE_FEATURES_PATH.exists(), "❌ node_features.parquet does not exist"

# ---- Load edges ----
edges = pd.read_parquet(EDGES_PATH)

required_cols = ["symbol_i", "symbol_j", "weight"]
assert list(edges.columns) == required_cols, (
    f"❌ edges_static.parquet columns must be {required_cols}, "
    f"found {list(edges.columns)}"
)

# ---- Dtype validation ----
assert edges["symbol_i"].dtype == object, "❌ symbol_i must be string/object"
assert edges["symbol_j"].dtype == object, "❌ symbol_j must be string/object"
assert edges["weight"].dtype == np.float32, "❌ weight must be float32"

# ---- Finiteness ----
assert np.isfinite(edges["weight"].to_numpy()).all(), "❌ Non-finite edge weights found"

# ---- Load node symbols for alignment ----
node_df = pd.read_parquet(NODE_FEATURES_PATH, columns=["symbol"])
node_symbols = set(node_df["symbol"].unique())

edge_symbols_i = set(edges["symbol_i"].unique())
edge_symbols_j = set(edges["symbol_j"].unique())

assert edge_symbols_i.issubset(node_symbols), (
    f"❌ symbol_i contains unknown symbols: {edge_symbols_i - node_symbols}"
)
assert edge_symbols_j.issubset(node_symbols), (
    f"❌ symbol_j contains unknown symbols: {edge_symbols_j - node_symbols}"
)

# ---- Baseline identity-graph semantics (v1 contract) ----
non_identity = edges[edges["symbol_i"] != edges["symbol_j"]]
assert non_identity.empty, (
    "❌ Non-identity edges found. "
    "Baseline graph must contain only self-loops."
)

non_unit_weight = edges[edges["weight"] != 1.0]
assert non_unit_weight.empty, (
    "❌ Edge weights must be exactly 1.0 for identity graph."
)

# ---- Final report ----
print("✅ edges_static.parquet structure and semantics are valid")
print(f"🔗 Total edges: {len(edges)}")
print(f"🧩 Identity graph confirmed (self-loops only)")

print("\n✅ Cell 4 PASSED — Graph contract satisfied.")
print("➡️ Proceed to Cell 5: HDF5 temporal sequence validation")


🧪 Running Cell 4: edges_static.parquet validation

✅ edges_static.parquet structure and semantics are valid
🔗 Total edges: 66
🧩 Identity graph confirmed (self-loops only)

✅ Cell 4 PASSED — Graph contract satisfied.
➡️ Proceed to Cell 5: HDF5 temporal sequence validation


In [9]:
# ============================================================
# Cell 5: HDF5 temporal sequence validation (lazy, memory-safe)
# ============================================================

from pathlib import Path
import h5py
import numpy as np
import pandas as pd

print("🧪 Running Cell 5: HDF5 temporal sequence validation\n")

PROJECT_ROOT = Path.cwd().parent
DATASET_DIR = PROJECT_ROOT / "datasets"
SCHEMA_PATH = DATASET_DIR / "feature_schema.txt"

# ---- Load schema ----
with open(SCHEMA_PATH, "r") as f:
    feature_schema = [line.rstrip("\n") for line in f.readlines()]
F = len(feature_schema)
SEQ_LEN = 60
TARGET_DIM = 2

H5_FILES = {
    "train": DATASET_DIR / "gnn_sequences_train.h5",
    "val":   DATASET_DIR / "gnn_sequences_val.h5",
    "test":  DATASET_DIR / "gnn_sequences_test.h5",
}

def validate_h5(split, path, sample_n=8):
    assert path.exists(), f"❌ {split}: HDF5 file missing"

    with h5py.File(path, "r") as h5:
        # ---- Required datasets ----
        for key in ["X", "y", "symbol", "date"]:
            assert key in h5, f"❌ {split}: Missing dataset '{key}'"

        X = h5["X"]
        y = h5["y"]
        sym = h5["symbol"]
        date = h5["date"]

        # ---- Shape checks ----
        assert X.ndim == 3, f"❌ {split}: X must be 3D"
        N, T, Fh5 = X.shape
        assert T == SEQ_LEN, f"❌ {split}: sequence length {T} != {SEQ_LEN}"
        assert Fh5 == F, f"❌ {split}: feature dim {Fh5} != {F}"

        assert y.shape == (N, TARGET_DIM), f"❌ {split}: y shape {y.shape} != (N, {TARGET_DIM})"
        assert sym.shape == (N,), f"❌ {split}: symbol shape invalid"
        assert date.shape == (N,), f"❌ {split}: date shape invalid"

        # ---- Dtype checks ----
        assert X.dtype == np.float32, f"❌ {split}: X dtype {X.dtype} != float32"
        assert y.dtype == np.float32, f"❌ {split}: y dtype {y.dtype} != float32"
        assert date.dtype == np.int64, f"❌ {split}: date dtype {date.dtype} != int64"

        # ---- Sampled finiteness checks (avoid full load) ----
        if N > 0:
            idx = np.linspace(0, N - 1, min(sample_n, N)).astype(int)
            X_sample = X[idx, :, :]
            y_sample = y[idx, :]
            assert np.isfinite(X_sample).all(), f"❌ {split}: Non-finite values in X sample"
            assert np.isfinite(y_sample).all(), f"❌ {split}: Non-finite values in y sample"

        # ---- Date reversibility ----
        try:
            _ = pd.to_datetime(date[:min(5, N)], unit="ns")
        except Exception as e:
            raise AssertionError(f"❌ {split}: date not convertible to datetime") from e

        print(f"✅ {split}: OK | N={N}, T={T}, F={Fh5}")

# ---- Run validation for each split ----
for split, path in H5_FILES.items():
    validate_h5(split, path)

print("\n✅ Cell 5 PASSED — All HDF5 temporal datasets satisfy the contract.")
print("➡️ Proceed to Cell 6: Cross-artifact consistency checks")


🧪 Running Cell 5: HDF5 temporal sequence validation

✅ train: OK | N=177703, T=60, F=263
✅ val: OK | N=38048, T=60, F=263
✅ test: OK | N=38149, T=60, F=263

✅ Cell 5 PASSED — All HDF5 temporal datasets satisfy the contract.
➡️ Proceed to Cell 6: Cross-artifact consistency checks


In [10]:
# ============================================================
# Cell 6: Cross-artifact consistency validation
# ============================================================

from pathlib import Path
import pandas as pd
import h5py
import numpy as np

print("🧪 Running Cell 6: Cross-artifact consistency checks\n")

PROJECT_ROOT = Path.cwd().parent
DATASET_DIR = PROJECT_ROOT / "datasets"

# ---- Paths ----
SCHEMA_PATH = DATASET_DIR / "feature_schema.txt"
NODE_FEATURES_PATH = DATASET_DIR / "node_features.parquet"
H5_PATHS = {
    "train": DATASET_DIR / "gnn_sequences_train.h5",
    "val":   DATASET_DIR / "gnn_sequences_val.h5",
    "test":  DATASET_DIR / "gnn_sequences_test.h5",
}

# ---- Load schema ----
with open(SCHEMA_PATH, "r") as f:
    feature_schema = [line.rstrip("\n") for line in f.readlines()]
F = len(feature_schema)

# ---- Load node feature symbols ----
node_df = pd.read_parquet(NODE_FEATURES_PATH, columns=["symbol"])
node_symbols = set(node_df["symbol"].unique())

assert len(node_symbols) > 0, "❌ No symbols found in node_features.parquet"

print(f"🔗 Node symbols: {len(node_symbols)}")

# ---- Validate each HDF5 split ----
split_symbols = {}

for split, path in H5_PATHS.items():
    with h5py.File(path, "r") as h5:
        X = h5["X"]
        sym = h5["symbol"]

        # Feature dimension consistency
        assert X.shape[2] == F, (
            f"❌ {split}: Feature dimension {X.shape[2]} != schema {F}"
        )

        # Symbol consistency
        symbols_in_split = set(s.decode("utf-8") if isinstance(s, bytes) else s for s in sym[:])
        unknown = symbols_in_split - node_symbols
        assert not unknown, (
            f"❌ {split}: Unknown symbols found: {unknown}"
        )

        split_symbols[split] = symbols_in_split

        print(f"✅ {split}: symbols={len(symbols_in_split)}, samples={X.shape[0]}")

# ---- Split isolation check (no leakage) ----
train_syms = split_symbols["train"]
val_syms   = split_symbols["val"]
test_syms  = split_symbols["test"]

assert train_syms == val_syms == test_syms, (
    "❌ Symbol sets differ across splits — potential leakage or filtering bug"
)

print("\n✅ All splits share identical symbol universe")

print("\n✅ Cell 6 PASSED — Cross-artifact consistency confirmed.")
print("➡️ Proceed to Cell 7: FINAL UNLOCK")


🧪 Running Cell 6: Cross-artifact consistency checks

🔗 Node symbols: 66
✅ train: symbols=66, samples=177703
✅ val: symbols=66, samples=38048
✅ test: symbols=66, samples=38149

✅ All splits share identical symbol universe

✅ Cell 6 PASSED — Cross-artifact consistency confirmed.
➡️ Proceed to Cell 7: FINAL UNLOCK


In [11]:
# ============================================================
# Cell 7: FINAL UNLOCK — Data Contracts Satisfied
# ============================================================

from datetime import datetime

print("🟢 MARKET SENTINEL — DATA CONTRACTS SATISFIED")
print("============================================")
print(f"Timestamp: {datetime.utcnow().isoformat()}Z\n")

print("✅ feature_schema.txt        — VALID")
print("✅ node_features.parquet     — VALID")
print("✅ edges_static.parquet      — VALID")
print("✅ gnn_sequences_train.h5    — VALID")
print("✅ gnn_sequences_val.h5      — VALID")
print("✅ gnn_sequences_test.h5     — VALID\n")

print("🔒 All data interfaces are consistent, finite, and schema-aligned.")
print("🚫 No preprocessing, inference, or mutation permitted downstream.")
print("➡️ DataLoader, Model, Training, and Backtesting are now AUTHORIZED.")

print("\n🏁 DATA COMPILATION COMPLETE — TRAINING UNLOCKED")


🟢 MARKET SENTINEL — DATA CONTRACTS SATISFIED
Timestamp: 2026-02-05T08:50:19.012138Z

✅ feature_schema.txt        — VALID
✅ node_features.parquet     — VALID
✅ edges_static.parquet      — VALID
✅ gnn_sequences_train.h5    — VALID
✅ gnn_sequences_val.h5      — VALID
✅ gnn_sequences_test.h5     — VALID

🔒 All data interfaces are consistent, finite, and schema-aligned.
🚫 No preprocessing, inference, or mutation permitted downstream.
➡️ DataLoader, Model, Training, and Backtesting are now AUTHORIZED.

🏁 DATA COMPILATION COMPLETE — TRAINING UNLOCKED


In [14]:
# ============================================================
# Cell 8: DataLoader smoke test (single sample)
# ============================================================

from pathlib import Path
import sys
import torch

# ---- Ensure project root is on PYTHONPATH ----
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from dataloader.temporal_dataloader import TemporalSequenceDataset

print("🧪 Running Cell 8: DataLoader smoke test\n")

DATASET_DIR = PROJECT_ROOT / "datasets"

H5_PATH = DATASET_DIR / "gnn_sequences_train.h5"
SCHEMA_PATH = DATASET_DIR / "feature_schema.txt"

# ---- Instantiate dataset ----
dataset = TemporalSequenceDataset(
    h5_path=H5_PATH,
    feature_schema_path=SCHEMA_PATH,
)

print(f"✅ Dataset instantiated | length = {len(dataset)}")

# ---- Pull one sample ----
sample = dataset[0]

X = sample["X"]
y = sample["y"]
symbol = sample["symbol"]
date = sample["date"]

# ---- Assertions ----
assert isinstance(X, torch.Tensor), "❌ X is not a torch.Tensor"
assert isinstance(y, torch.Tensor), "❌ y is not a torch.Tensor"

assert X.shape[0] == 60, "❌ X sequence length != 60"
assert X.ndim == 2, "❌ X must be [60, F]"
assert y.shape == (2,), "❌ y must be shape (2,)"

assert X.dtype == torch.float32, "❌ X dtype != float32"
assert y.dtype == torch.float32, "❌ y dtype != float32"

assert isinstance(symbol, str), "❌ symbol must be str"
assert hasattr(date, "to_pydatetime"), "❌ date must be datetime-like"

print("📌 Sample inspection:")
print(f"  Symbol: {symbol}")
print(f"  Date:   {date}")
print(f"  X shape: {tuple(X.shape)} | y: {y.tolist()}")

# ---- Cleanup ----
dataset.close()

print("\n✅ Cell 8 PASSED — DataLoader is contract-compliant.")
print("➡️ Proceed to Model definition phase")


🧪 Running Cell 8: DataLoader smoke test

✅ Dataset instantiated | length = 177703
📌 Sample inspection:
  Symbol: AAPL
  Date:   2015-03-02 00:00:00
  X shape: (60, 263) | y: [0.002091564005240798, 0.0]

✅ Cell 8 PASSED — DataLoader is contract-compliant.
➡️ Proceed to Model definition phase


In [15]:
# ============================================================
# Cell 9: TemporalEncoder smoke test
# ============================================================

from pathlib import Path
import sys
import torch

# ---- Ensure project root on PYTHONPATH ----
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from dataloader.temporal_dataloader import TemporalSequenceDataset
from models.temporal_encoder import TemporalEncoder

print("🧪 Running Cell 9: TemporalEncoder smoke test\n")

DATASET_DIR = PROJECT_ROOT / "datasets"

H5_PATH = DATASET_DIR / "gnn_sequences_train.h5"
SCHEMA_PATH = DATASET_DIR / "feature_schema.txt"

# ---- Load feature schema ----
with open(SCHEMA_PATH, "r") as f:
    feature_schema = [line.rstrip("\n") for line in f.readlines()]

F = len(feature_schema)
H = 128  # hidden dimension for smoke test (arbitrary but fixed)

# ---- Instantiate dataset and pull a mini-batch ----
dataset = TemporalSequenceDataset(
    h5_path=H5_PATH,
    feature_schema_path=SCHEMA_PATH,
)

# Create a small batch manually (no DataLoader yet)
batch_size = 4
X_batch = torch.stack([dataset[i]["X"] for i in range(batch_size)], dim=0)

print(f"📐 Input batch shape: {tuple(X_batch.shape)}")

# ---- Instantiate TemporalEncoder ----
encoder = TemporalEncoder(
    input_dim=F,
    hidden_dim=H,
)

# ---- Forward pass ----
with torch.no_grad():
    H_out = encoder(X_batch)

# ---- Assertions ----
assert isinstance(H_out, torch.Tensor), "❌ Output is not a torch.Tensor"
assert H_out.shape == (batch_size, H), f"❌ Output shape {H_out.shape} != {(batch_size, H)}"
assert H_out.dtype == torch.float32, "❌ Output dtype != float32"
assert torch.isfinite(H_out).all(), "❌ Non-finite values in encoder output"

print("📌 TemporalEncoder output inspection:")
print(f"  Output shape: {tuple(H_out.shape)}")
print(f"  Dtype: {H_out.dtype}")
print(f"  Sample (first row, first 5 vals): {H_out[0, :5].tolist()}")

# ---- Cleanup ----
dataset.close()

print("\n✅ Cell 9 PASSED — TemporalEncoder is contract-compliant.")
print("➡️ Proceed to GraphEncoder implementation")


🧪 Running Cell 9: TemporalEncoder smoke test

📐 Input batch shape: (4, 60, 263)
📌 TemporalEncoder output inspection:
  Output shape: (4, 128)
  Dtype: torch.float32
  Sample (first row, first 5 vals): [-0.9228795170783997, 0.9436871409416199, 0.7474874258041382, -0.42305558919906616, 0.9453398585319519]

✅ Cell 9 PASSED — TemporalEncoder is contract-compliant.
➡️ Proceed to GraphEncoder implementation


In [16]:
# ============================================================
# Cell 10: GraphEncoder smoke test
# ============================================================

from pathlib import Path
import sys
import torch

# ---- Ensure project root on PYTHONPATH ----
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from dataloader.temporal_dataloader import TemporalSequenceDataset
from models.temporal_encoder import TemporalEncoder
from models.graph_encoder import GraphEncoder

print("🧪 Running Cell 10: GraphEncoder smoke test\n")

DATASET_DIR = PROJECT_ROOT / "datasets"
H5_PATH = DATASET_DIR / "gnn_sequences_train.h5"
SCHEMA_PATH = DATASET_DIR / "feature_schema.txt"

# ---- Load schema ----
with open(SCHEMA_PATH, "r") as f:
    feature_schema = [line.rstrip("\n") for line in f.readlines()]

F = len(feature_schema)
H = 128
BATCH_SIZE = 4

# ---- Dataset + batch ----
dataset = TemporalSequenceDataset(
    h5_path=H5_PATH,
    feature_schema_path=SCHEMA_PATH,
)

X_batch = torch.stack([dataset[i]["X"] for i in range(BATCH_SIZE)], dim=0)

# ---- Temporal encoding ----
temporal_encoder = TemporalEncoder(input_dim=F, hidden_dim=H)
with torch.no_grad():
    H_temporal = temporal_encoder(X_batch)

print(f"📐 Temporal embedding shape: {tuple(H_temporal.shape)}")

# ---- Graph encoding ----
graph_encoder = GraphEncoder(hidden_dim=H)
with torch.no_grad():
    H_graph = graph_encoder(H_temporal)

# ---- Assertions ----
assert isinstance(H_graph, torch.Tensor), "❌ Output is not a torch.Tensor"
assert H_graph.shape == (BATCH_SIZE, H), f"❌ Output shape {H_graph.shape} != {(BATCH_SIZE, H)}"
assert H_graph.dtype == torch.float32, "❌ Output dtype != float32"
assert torch.isfinite(H_graph).all(), "❌ Non-finite values in GraphEncoder output"

# Identity check
assert torch.allclose(H_graph, H_temporal), "❌ GraphEncoder is not identity"

print("📌 GraphEncoder output inspection:")
print(f"  Output shape: {tuple(H_graph.shape)}")
print(f"  Identity mapping confirmed")

# ---- Cleanup ----
dataset.close()

print("\n✅ Cell 10 PASSED — GraphEncoder is contract-compliant.")
print("➡️ Proceed to FusionLayer implementation")


🧪 Running Cell 10: GraphEncoder smoke test

📐 Temporal embedding shape: (4, 128)
📌 GraphEncoder output inspection:
  Output shape: (4, 128)
  Identity mapping confirmed

✅ Cell 10 PASSED — GraphEncoder is contract-compliant.
➡️ Proceed to FusionLayer implementation


In [17]:
# ============================================================
# Cell 11: FusionLayer smoke test
# ============================================================

from pathlib import Path
import sys
import torch

# ---- Ensure project root on PYTHONPATH ----
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from dataloader.temporal_dataloader import TemporalSequenceDataset
from models.temporal_encoder import TemporalEncoder
from models.graph_encoder import GraphEncoder
from models.fusion import FusionLayer

print("🧪 Running Cell 11: FusionLayer smoke test\n")

DATASET_DIR = PROJECT_ROOT / "datasets"
H5_PATH = DATASET_DIR / "gnn_sequences_train.h5"
SCHEMA_PATH = DATASET_DIR / "feature_schema.txt"

# ---- Load schema ----
with open(SCHEMA_PATH, "r") as f:
    feature_schema = [line.rstrip("\n") for line in f.readlines()]

F = len(feature_schema)
H = 128
BATCH_SIZE = 4

# ---- Dataset + batch ----
dataset = TemporalSequenceDataset(
    h5_path=H5_PATH,
    feature_schema_path=SCHEMA_PATH,
)

X_batch = torch.stack([dataset[i]["X"] for i in range(BATCH_SIZE)], dim=0)

# ---- Temporal encoding ----
temporal_encoder = TemporalEncoder(input_dim=F, hidden_dim=H)
with torch.no_grad():
    H_temporal = temporal_encoder(X_batch)

# ---- Graph encoding ----
graph_encoder = GraphEncoder(hidden_dim=H)
with torch.no_grad():
    H_graph = graph_encoder(H_temporal)

print(f"📐 Graph embedding shape: {tuple(H_graph.shape)}")

# ---- Fusion ----
fusion = FusionLayer(hidden_dim=H)
with torch.no_grad():
    H_fused = fusion(H_graph)

# ---- Assertions ----
assert isinstance(H_fused, torch.Tensor), "❌ Output is not a torch.Tensor"
assert H_fused.shape == (BATCH_SIZE, H), f"❌ Output shape {H_fused.shape} != {(BATCH_SIZE, H)}"
assert H_fused.dtype == torch.float32, "❌ Output dtype != float32"
assert torch.isfinite(H_fused).all(), "❌ Non-finite values in FusionLayer output"

# Identity check
assert torch.allclose(H_fused, H_graph), "❌ FusionLayer is not identity"

print("📌 FusionLayer output inspection:")
print(f"  Output shape: {tuple(H_fused.shape)}")
print(f"  Identity mapping confirmed")

# ---- Cleanup ----
dataset.close()

print("\n✅ Cell 11 PASSED — FusionLayer is contract-compliant.")
print("➡️ Proceed to full model wiring")


🧪 Running Cell 11: FusionLayer smoke test

📐 Graph embedding shape: (4, 128)
📌 FusionLayer output inspection:
  Output shape: (4, 128)
  Identity mapping confirmed

✅ Cell 11 PASSED — FusionLayer is contract-compliant.
➡️ Proceed to full model wiring


In [18]:
# ============================================================
# Cell 12: Full Model forward smoke test
# ============================================================

from pathlib import Path
import sys
import torch

# ---- Ensure project root on PYTHONPATH ----
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from dataloader.temporal_dataloader import TemporalSequenceDataset
from models.model import MarketSentinelModel

print("🧪 Running Cell 12: Full model forward smoke test\n")

DATASET_DIR = PROJECT_ROOT / "datasets"
H5_PATH = DATASET_DIR / "gnn_sequences_train.h5"
SCHEMA_PATH = DATASET_DIR / "feature_schema.txt"

# ---- Load schema ----
with open(SCHEMA_PATH, "r") as f:
    feature_schema = [line.rstrip("\n") for line in f.readlines()]

F = len(feature_schema)
H = 128
BATCH_SIZE = 4

# ---- Dataset + batch ----
dataset = TemporalSequenceDataset(
    h5_path=H5_PATH,
    feature_schema_path=SCHEMA_PATH,
)

X_batch = torch.stack([dataset[i]["X"] for i in range(BATCH_SIZE)], dim=0)

print(f"📐 Input batch shape: {tuple(X_batch.shape)}")

# ---- Model ----
model = MarketSentinelModel(
    input_dim=F,
    hidden_dim=H,
)

model.eval()

with torch.no_grad():
    y_hat = model(X_batch)

# ---- Assertions ----
assert isinstance(y_hat, torch.Tensor), "❌ Output is not a torch.Tensor"
assert y_hat.shape == (BATCH_SIZE, 2), f"❌ Output shape {y_hat.shape} != {(BATCH_SIZE, 2)}"
assert y_hat.dtype == torch.float32, "❌ Output dtype != float32"
assert torch.isfinite(y_hat).all(), "❌ Non-finite values in model output"

print("📌 Model output inspection:")
print(f"  Output shape: {tuple(y_hat.shape)}")
print(f"  Sample predictions:\n{y_hat}")

# ---- Cleanup ----
dataset.close()

print("\n✅ Cell 12 PASSED — Full model is contract-compliant.")
print("🏁 MODEL STACK COMPLETE — READY FOR TRAINING")


🧪 Running Cell 12: Full model forward smoke test

📐 Input batch shape: (4, 60, 263)
📌 Model output inspection:
  Output shape: (4, 2)
  Sample predictions:
tensor([[ 0.4423, -0.1023],
        [ 0.4458, -0.2671],
        [ 0.4327, -0.3885],
        [ 0.4520, -0.4497]])

✅ Cell 12 PASSED — Full model is contract-compliant.
🏁 MODEL STACK COMPLETE — READY FOR TRAINING


In [22]:
# ============================================================
# Cell 13: Training dry-run test (1 batch, 1 step)
# ============================================================

from pathlib import Path
import sys
import torch
import torch.nn as nn

# ---- Ensure project root on PYTHONPATH ----
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from dataloader.temporal_dataloader import TemporalSequenceDataset
from models.model import MarketSentinelModel

print("🧪 Running Cell 13: Training dry-run test\n")

# ---- Device ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥 Device: {device}")

# ---- Paths ----
DATASET_DIR = PROJECT_ROOT / "datasets"
H5_PATH = DATASET_DIR / "gnn_sequences_train.h5"
SCHEMA_PATH = DATASET_DIR / "feature_schema.txt"

# ---- Load schema ----
with open(SCHEMA_PATH, "r") as f:
    feature_schema = [line.rstrip("\n") for line in f.readlines()]

F = len(feature_schema)
H = 128
BATCH_SIZE = 4  # very safe for 4GB VRAM

# ---- Dataset ----
dataset = TemporalSequenceDataset(
    h5_path=H5_PATH,
    feature_schema_path=SCHEMA_PATH,
)

# ---- Build one batch ----
X_batch = torch.stack([dataset[i]["X"] for i in range(BATCH_SIZE)], dim=0)
y_batch = torch.stack([dataset[i]["y"] for i in range(BATCH_SIZE)], dim=0)

X_batch = X_batch.to(device)
y_batch = y_batch.to(device)

print(f"📐 X batch: {tuple(X_batch.shape)} | y batch: {tuple(y_batch.shape)}")

# ---- Model ----
model = MarketSentinelModel(
    input_dim=F,
    hidden_dim=H,
).to(device)

model.train()

# ---- Optimizer & loss ----
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-3,
    weight_decay=1e-4,
)

loss_fn = nn.MSELoss()

# ---- Forward ----
y_hat = model(X_batch)

assert y_hat.shape == (BATCH_SIZE, 2), "❌ Model output shape incorrect"
assert torch.isfinite(y_hat).all(), "❌ Non-finite model output"

# ---- Loss ----
loss = loss_fn(y_hat[:, 0], y_batch[:, 0]) + loss_fn(y_hat[:, 1], y_batch[:, 1])
print(f"📉 Dry-run loss: {loss.item():.6f}")

# ---- Backward ----
optimizer.zero_grad()
loss.backward()
optimizer.step()

print("✅ Backward + optimizer step completed")

# ---- Cleanup ----
dataset.close()
torch.cuda.empty_cache()

print("\n🟢 Cell 13 PASSED — Training dry-run successful.")
print("➡️ You may now run training/train.py safely")


🧪 Running Cell 13: Training dry-run test

🖥 Device: cuda
📐 X batch: (4, 60, 263) | y batch: (4, 2)
📉 Dry-run loss: 0.181352
✅ Backward + optimizer step completed

🟢 Cell 13 PASSED — Training dry-run successful.
➡️ You may now run training/train.py safely


In [10]:
# ============================================================
# Cell 15: Checkpoint integrity smoke test
# ============================================================

from pathlib import Path
import torch

print("🧪 Running Cell 15: Checkpoint integrity check\n")

PROJECT_ROOT = Path.cwd().parent
CHECKPOINT_PATH = PROJECT_ROOT / "checkpoints" / "model_best.pt"

# ---- Existence check ----
assert CHECKPOINT_PATH.exists(), (
    f"❌ Checkpoint not found at {CHECKPOINT_PATH}"
)

file_size_mb = CHECKPOINT_PATH.stat().st_size / (1024 * 1024)
print(f"📦 Checkpoint file found: {CHECKPOINT_PATH.name}")
print(f"📏 File size: {file_size_mb:.2f} MB")

# assert file_size_mb > 1.0, "❌ Checkpoint file too small — likely invalid"

# ---- Load checkpoint ----
ckpt = torch.load(CHECKPOINT_PATH, map_location="cpu")

print("\n✅ Checkpoint loaded successfully")

# ---- Key validation ----
EXPECTED_KEYS = {
    "model_state_dict",
    "epoch",
    "val_loss",
    "input_dim",
    "hidden_dim",
}

missing_keys = EXPECTED_KEYS - set(ckpt.keys())
assert not missing_keys, f"❌ Missing keys in checkpoint: {missing_keys}"

print("🔑 Checkpoint keys verified:")
for k in ckpt.keys():
    print(f"  - {k}")

# ---- Value sanity checks ----
assert isinstance(ckpt["epoch"], int), "❌ epoch is not int"
assert isinstance(ckpt["val_loss"], float), "❌ val_loss is not float"
assert ckpt["input_dim"] > 0, "❌ input_dim invalid"
assert ckpt["hidden_dim"] > 0, "❌ hidden_dim invalid"

print("\n📌 Checkpoint metadata:")
print(f"  Saved epoch:     {ckpt['epoch']}")
print(f"  Validation loss: {ckpt['val_loss']:.6f}")
print(f"  Input dim (F):   {ckpt['input_dim']}")
print(f"  Hidden dim (H):  {ckpt['hidden_dim']}")

# ---- State dict sanity ----
state_dict = ckpt["model_state_dict"]
assert isinstance(state_dict, dict), "❌ model_state_dict is not a dict"
assert len(state_dict) > 0, "❌ model_state_dict is empty"

sample_key = next(iter(state_dict))
sample_tensor = state_dict[sample_key]

assert torch.is_tensor(sample_tensor), "❌ State dict values are not tensors"
assert torch.isfinite(sample_tensor).all(), "❌ Non-finite values in checkpoint tensor"

print("\n📊 State dict sanity check:")
print(f"  Number of tensors: {len(state_dict)}")
print(f"  Sample key:        {sample_key}")
print(f"  Sample shape:     {tuple(sample_tensor.shape)}")
print(f"  Sample dtype:     {sample_tensor.dtype}")

print("\n🟢 Cell 15 PASSED — Checkpoint is complete, valid, and loadable.")
print("➡️ The model was saved correctly. No corruption detected.")

# # ============================================================
# # Inspect actual contents of model_best.pt
# # ============================================================

# from pathlib import Path
# import torch

# PROJECT_ROOT = Path.cwd().parent
# CHECKPOINT_PATH = PROJECT_ROOT / "checkpoints" / "model_best.pt"

# ckpt = torch.load(CHECKPOINT_PATH, map_location="cpu")

# print("Type of checkpoint object:", type(ckpt))

# if isinstance(ckpt, dict):
#     print("\nKeys found in checkpoint:")
#     for k in ckpt.keys():
#         print("  -", k)
# else:
#     print("\nCheckpoint is NOT a dict.")
#     print("Likely contains a raw state_dict or model.")


🧪 Running Cell 15: Checkpoint integrity check

📦 Checkpoint file found: model_best.pt
📏 File size: 0.58 MB

✅ Checkpoint loaded successfully
🔑 Checkpoint keys verified:
  - model_state_dict
  - epoch
  - val_loss
  - input_dim
  - hidden_dim

📌 Checkpoint metadata:
  Saved epoch:     3
  Validation loss: 0.001136
  Input dim (F):   263
  Hidden dim (H):  128

📊 State dict sanity check:
  Number of tensors: 6
  Sample key:        temporal_encoder.gru.weight_ih_l0
  Sample shape:     (384, 263)
  Sample dtype:     torch.float32

🟢 Cell 15 PASSED — Checkpoint is complete, valid, and loadable.
➡️ The model was saved correctly. No corruption detected.


In [ ]:
# ============================================================
# Cell 14: Backtesting smoke test (file integrity verification)
# ============================================================
import h5py
import numpy as np
import tempfile
import shutil

def make_canary_h5(
    src_h5: Path,
    fraction: float,
) -> Path:
    assert 0 < fraction <= 1.0

    tmp_dir = Path(tempfile.mkdtemp())
    dst_h5 = tmp_dir / src_h5.name

    with h5py.File(src_h5, "r") as src, h5py.File(dst_h5, "w") as dst:
        n = src["X"].shape[0]
        k = max(1, int(n * fraction))

        for key in src.keys():
            dst.create_dataset(
                key,
                data=src[key][:k],
                dtype=src[key].dtype,
            )

    return dst_h5

from pathlib import Path
import sys
import json

# ---- Ensure project root on PYTHONPATH ----
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("🧪 Running Cell 14: Backtesting smoke test\n")

print("⚠️  FAIR WARNING")
print("--------------------------------------------------")
print("This is a STRUCTURAL smoke test only.")
print("Any metrics / IC values printed here are:")
print("  - NOT statistically meaningful")
print("  - NOT strategy-evaluated")
print("  - NOT suitable for financial conclusions")
print("--------------------------------------------------\n")

from backtesting.backtest import backtest

# ---- Paths ----
DATASET_DIR = PROJECT_ROOT / "datasets"
CHECKPOINT_DIR = PROJECT_ROOT / "checkpoints"
BACKTEST_DIR = PROJECT_ROOT / "backtests"

# TEST_H5 = DATASET_DIR / "gnn_sequences_test.h5"
FULL_TEST_H5 = DATASET_DIR / "gnn_sequences_test.h5"
TEST_H5 = make_canary_h5(FULL_TEST_H5, fraction=0.05)

print(f"🧪 Canary test HDF5 created → {TEST_H5}")

SCHEMA_PATH = DATASET_DIR / "feature_schema.txt"
CKPT_PATH = CHECKPOINT_DIR / "model_best.pt"

# ---- Sanity assertions (before execution) ----
assert TEST_H5.exists(), "❌ Test HDF5 missing"
assert SCHEMA_PATH.exists(), "❌ feature_schema.txt missing"
assert CKPT_PATH.exists(), "❌ model_best.pt missing"

print("✅ Required inputs present")
print(f"  - Test HDF5: {TEST_H5.name}")
print(f"  - Schema:   {SCHEMA_PATH.name}")
print(f"  - Checkpt:  {CKPT_PATH.name}\n")

# ---- Execute backtest ----
backtest(
    test_h5=TEST_H5,
    feature_schema_path=SCHEMA_PATH,
    checkpoint_path=CKPT_PATH,
)

# ---- Verify outputs ----
EXPECTED_OUTPUTS = [
    BACKTEST_DIR / "predictions.parquet",
    BACKTEST_DIR / "metrics.json",
    BACKTEST_DIR / "ic_timeseries.parquet",
    BACKTEST_DIR / "signal_confidence.json",
]

missing = [p.name for p in EXPECTED_OUTPUTS if not p.exists()]
assert not missing, f"❌ Missing backtest outputs: {missing}"

print("\n✅ All expected backtest artifacts created:\n")
for p in EXPECTED_OUTPUTS:
    print(f"  - {p.name}")

# ---- Load & lightly inspect signal confidence ----
with open(BACKTEST_DIR / "signal_confidence.json", "r") as f:
    signal_conf = json.load(f)

print("\n📌 Signal confidence structure check:")
for horizon, stats in signal_conf.items():
    print(f"  {horizon}:")
    for k, v in stats.items():
        print(f"    - {k}: {v}")

print("\n🟢 Cell 14 PASSED — Backtesting pipeline is structurally sound.")
print("➡️ You may now trust backtesting/backtest.py as a verified artifact.")


🧪 Running Cell 14: Backtesting smoke test

⚠️  FAIR WARNING
--------------------------------------------------
This is a STRUCTURAL smoke test only.
Any metrics / IC values printed here are:
  - NOT statistically meaningful
  - NOT strategy-evaluated
  - NOT suitable for financial conclusions
--------------------------------------------------

✅ Required inputs present
  - Test HDF5: gnn_sequences_test.h5
  - Schema:   feature_schema.txt
  - Checkpt:  model_best.pt

🖥 Using device: cuda


KeyboardInterrupt: 